# Imports

In [14]:
from utils import *
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import pickle
from datetime import datetime as dt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import keras_tuner
import warnings
import calendar
import collections.abc
collections.Iterable = collections.abc.Iterable
warnings.filterwarnings("ignore")

In [15]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Models Setup

In [16]:
models = {}

## Deeper Model

In [17]:
X = pd.concat([exog, train], axis=1)
Y = X.shift(-1, fill_value=0)
X_train, X_test, y_train, y_test , X_test_copy, X_test_index = data_prep(X, Y)

(1515, 1808, 1)
(169, 1808, 1)
(1515, 1808, 1)
(169, 1808, 1)


In [26]:
model1 = keras.models.Sequential([
    keras.layers.LSTM(50, return_sequences=True, input_shape=[X_train.shape[1], X_train.shape[2]]),
    keras.layers.LSTM(50, return_sequences=True),
    keras.layers.LSTM(50, return_sequences=True),
    keras.layers.LSTM(50, return_sequences=True),
    keras.layers.LSTM(50, return_sequences=True),
    keras.layers.LSTM(50, return_sequences=True),
    keras.layers.Dense(1, activation='relu')
])
models['model1'] = {'model': model1,
                    'params': {}
                    }
models['model2'] = {'model': 'funk',
                    'params': {}
                    }

In [29]:
for i in models:
    print(models[i]['model'])

funk


In [28]:
models['model2']['model']

'funk'

## Implementing Layer Normalization

In [ ]:
X = pd.concat([exog, train], axis=1)
Y = X.shift(-1, fill_value=0)
X_train, X_test, y_train, y_test , X_test_copy, X_test_index = data_prep(X, Y)

In [ ]:
lstm = keras.models.Sequential([
    keras.layers.RNN(tfa.rnn.LayerNormLSTMCell(100, dropout=0.1, recurrent_dropout=0.1), return_sequences=True, input_shape=[X_train.shape[1], X_train.shape[2]]),
    keras.layers.Dense(1, activation='relu')
])
pred = predict(lstm, X_train, y_train, X_test, X_test_copy, X_test_index)

In [ ]:
compare(X_test_copy.iloc[:, -1792:], pred)

# Hypertuning

In [ ]:
X = train
Y = train.shift(-1, fill_value=0)
X_train, X_test, y_train, y_test, X_test_copy, X_test_index = data_prep(X, Y)
X_train_mini = X_train[:150, :]
y_train_mini = y_train[:150, :]
X_val_mini = X_train[151:160, :]
y_val_mini = y_train[151:160, :]

In [ ]:
def dummy_model(hp):
    lstm = keras.models.Sequential([
    keras.layers.LSTM(hp.Choice('units', [i*10 for i in range(1, 201)]), return_sequences=True, input_shape=[X_train.shape[1], X_train.shape[2]]),
    keras.layers.Dense(1, activation='relu')
    ])
    lstm.compile(loss=keras.losses.MeanSquaredLogarithmicError(), optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=np.linspace(0.0, 0.01, 50).tolist())))
    return lstm

In [ ]:
tuner = keras_tuner.RandomSearch(dummy_model, objective='val_loss', max_trials=200)
tuner.search(X_train_mini, y_train_mini, epochs=10, validation_data=(X_val_mini, y_val_mini))
tuner.get_best_hyperparameters()

In [ ]:
model1 = {}
model2 = {}
model3 = {}

tuning1 = {
    "batch_size": [32, 64, 128],
    "epochs": [10, 20, 30],
    "optimizer": ["Adam", "SGD"],
}
tuning2 = {}
tuning3 = {}

experiment_loop(
    models=[model1, model2, model3],
    tuning=[tuning1, tuning2, tuning3],
)


def test_model_valid():
    # test model is valid
    # test all param choices
    # training and prediction forward and backward pass run correctly
    pass

def _tune():
    pass 

def _train_full_set():
    # verbose off
    pass

def _predict():
    pass

def _plot_losses():
    # loss comparison between models
    pass

def _plot_y_pred_vs_y():
    # comparing a massively reduces subset of data
    # comparing models
    pass

def experiment_loop(
    models: list,
    tuning: list,
):
    x_data = pd.DataFrame()
    y_data = pd.DataFrame()


    losses = [] # list[list]
    boards = []
    y_preds = []
    pred_metrics = []
    for i, model in enumerate(models):
        # 1. tune
        # reduce data size for tuning
        # hyperparam tune yields best params for each model
        _tune(model, tuning[i])
        print(f"best model params for {model_name} are {best_params}")

        # 2. train
        trained_model, loss_arr = _train_full_set(model, best_params)
        losses.append(loss_arr)
        other_metric = {
            "accuracy": 0.9,
            "r2": 0.8,
        }
        other_metrics.append(other_metric)
        boards.append(board_path)

        # 3. predict
        y_pred = _predict(trained_model, x_holdout)
        y_preds.append(y_pred)
        pred_metric = {
            "accuracy": 0.9,
            "r2": 0.8,
        }
        pred_metrics.append(pred_metric)

    _plot_losses(losses)
    _plot_metrics(other_metrics)
    _plot_y_pred_vs_y(y_preds)